In [ ]:
import os
import urllib.request
from multiprocessing.pool import ThreadPool as Pool
import utils.request as request
import utils.const as const

In [ ]:
db=const.grpId
collection=const.PostCollection
directory='attachment'


failedFile='failed.txt'
failed=[]
prefix='https://graph.facebook.com/v9.0/'
pool_size=1200  ## directly proportional to network bandwidth

In [ ]:
token=[]
# token.append('insert Your token')
# token.append('more the number of token will result in smaller execution time' )
token.append(const.tokenPost)
token.append(const.tokenComments)
token.append(const.tokenSubComments)
token.append(const.tokenAttachments)
token.append(const.tokenCommentAttachments)
token.append(const.tokenCommentLikeCount)
token.append(const.tokenPostReactions)

In [ ]:
postId={}
commentId={}
with open(failedFile,'r') as fp:
  for line in fp.readlines():
    id=line.split(' ')[0]
    arr=id.split('_')
    if len(arr)==3:
      postId[arr[0]+'_'+arr[1]]=1
    else:
      commentId[arr[0]+'_'+arr[1]]=arr[2]

print(len(postId))
print(len(commentId))

In [ ]:
def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

In [ ]:
def postAttachment(postId,token):
  url=prefix+postId+"?fields=attachments&access_token="+token
  print(url)
  r=request.apiCall(url)
  images=json_extract(r,'src')
  videos=json_extract(r,'source')
  print(len(images))
  print(len(videos))

  for i in range(len(images)):
    fileName=postId+'_I'+str(i)
    c=0
    while c<5:
      try:
        urllib.request.urlretrieve(images[i], os.path.join(directory,fileName))
        print(fileName)
        break
      except Exception as e:
        print(e)
        c+=1
    if c>=5:
      failed.append([fileName,images[i]])

  for i in range(len(videos)):
    fileName=postId+'_V'+str(i)
    c=0
    while c<5:
      try:
        urllib.request.urlretrieve(videos[i], os.path.join(directory,fileName))
        print(fileName)
        break
      except Exception as e:
        print(e)
        c+=1
    if c>=5:
      failed.append([fileName,videos[i]])

In [ ]:
def commentAttachment(postId,commentId,token):
  url=prefix+commentId+"?fields=attachment&access_token="+token
  print(url)
  r=request.apiCall(url)
  images=json_extract(r,'src')
  videos=json_extract(r,'source')
  print(len(images))
  print(len(videos))

  for i in range(len(images)):
    fileName=postId+'_'+commentId+'_I'+str(i)
    c=0
    while c<5:
      try:
        urllib.request.urlretrieve(images[i], os.path.join(directory,fileName))
        print(fileName)
        break
      except Exception as e:
        print(e)
        c+=1
    if c>=5:
      failed.append([fileName,images[i]])

  for i in range(len(videos)):
    fileName=postId+'_'+commentId+'_V'+str(i)
    c=0
    while c<5:
      try:
        urllib.request.urlretrieve(videos[i], os.path.join(directory,fileName))
        print(fileName)
        break
      except Exception as e:
        print(e)
        c+=1
    if c>=5:
      failed.append([fileName,videos[i]])

In [ ]:
pool = Pool(pool_size)

In [ ]:
i=0
for k,v in postId.items():
  pool.apply_async(postAttachment,(k,token[i%len(token)]))
  i+=1


In [ ]:
i=0
for k,v in commentId.items():
  pool.apply_async(commentAttachment,(k,v,token[i%len(token)]))
  i+=1


In [ ]:
pool.close()
pool.join()

In [ ]:
print(len(failed))

In [ ]:
if len(failed):
    print('Now this is some serious trouble.... see if u can ignore those many attachments?')
else:
    print('Done')